In [102]:
import numpy as np
import scipy.linalg as linalg
from dtuimldmtools import similarity
import matplotlib.pyplot as plt
from scipy.linalg import svd
from pandas.plotting import scatter_matrix
import pandas as pd
import numpy as np
import sklearn.linear_model as lm
from matplotlib.pyplot import figure, legend, plot, show, xlabel, ylabel
# exercise 8.1.1
from dtuimldmtools import dbplotf, train_neural_net, visualize_decision_boundary

import importlib_resources
import numpy as np
import sklearn.linear_model as lm
from matplotlib.pylab import (
    figure,
    grid,
    legend,
    loglog,
    semilogx,
    show,
    subplot,
    title,
    xlabel,
    ylabel,
)
from scipy.io import loadmat
from sklearn import model_selection

from dtuimldmtools import rlr_validate
import numpy as np
import scipy.linalg as linalg
from dtuimldmtools import similarity
import matplotlib.pyplot as plt
from scipy.linalg import svd
from pandas.plotting import scatter_matrix
import pandas as pd
import numpy as np
import sklearn.linear_model as lm
from matplotlib.pyplot import figure, legend, plot, show, xlabel, ylabel
# exercise 8.1.1
from dtuimldmtools import dbplotf, train_neural_net, visualize_decision_boundary
import importlib_resources
import numpy as np
import torch
from matplotlib.pyplot import figure, show, title
from scipy.io import loadmat
import importlib_resources
import numpy as np
import sklearn.linear_model as lm
from matplotlib.pylab import (
    figure,
    grid,
    legend,
    loglog,
    semilogx,
    show,
    subplot,
    title,
    xlabel,
    ylabel,
)
from scipy.io import loadmat
from sklearn import model_selection

from dtuimldmtools import rlr_validate

# fetch dataset 
# wine = fetch_ucirepo(id=109) 
  
# # data (as pandas dataframes) 
# X = wine.data.features 
# y = wine.data.targets 

# totaldata= (wine.data)
  
# metadata 
# print(wine.metadata) 
  
# variable information 
# print(wine.variables) 

# OFFLINE LOADING OF DATA
X = np.loadtxt('../wine/wine.data', delimiter=',')

#Extract classes from X
y = X[:,0]
X = np.delete(X,0,axis=1)
# y = np.loadtxt('../wine/wine.names', delimiter=',')

Xorig = X
# Standardizing the data

X = (X - np.mean(X, axis=0)) / np.std(X, axis=0)



attributeNames = [
    "Alcohol",
    "Malic acid",
    "Ash",
    "Alcalinity of ash",
    "Magnesium",
    "Total phenols",
    "Flavanoids",
    "Nonflavanoid phenols",
    "Proanthocyanins",
    "Color intensity",
    "Hue",
    "OD280/OD315 of diluted wines",
    "Proline"
]

N, M = X.shape

# Add offset attribute
attributeNames = ["Offset"] + attributeNames
# M = M + 1


# Convert class labels to 0, 1, 2
y = y - 1

classNames = ["1", "2", "3"]


In [ ]:
# %% Model fitting and prediction using logistic regression

## Crossvalidation
# Create OUTER crossvalidation partition for evaluation
K = 10
CV = model_selection.KFold(K, shuffle=True, random_state=42)
# CV = model_selection.KFold(K, shuffle=False)

# Values of lambda
lambdas = np.logspace(0, 3, 100)
# Initialize variables
# T = len(lambdas)
Error_train = np.empty((K, 1))
Error_test = np.empty((K, 1))
Error_train_rlr = np.empty((K, 1))
Error_test_rlr = np.empty((K, 1))
Error_train_nofeatures = np.empty((K, 1))
Error_test_nofeatures = np.empty((K, 1))
w_rlr = np.empty((M, K))
mu = np.empty((K, M - 1))
sigma = np.empty((K, M - 1))
w_noreg = np.empty((M, K))

k = 0
for train_index, test_index in CV.split(X, y):
    Error_train_avg = []
    Error_test_avg = []
    # extract training and test set for current CV fold
    X_train = X[train_index]
    y_train = y[train_index]
    X_test = X[test_index]
    y_test = y[test_index]
    internal_cross_validation = 10
    
    internal_cv = model_selection.KFold(internal_cross_validation, shuffle=True, random_state=42)

    # Inner cross-validation to find the best regularization parameter (lambda)
    best_lambda = 0.0
    min_error = float('inf')
    for train_inner_idx, val_inner_idx in internal_cv.split(X_train, y_train):
        
        #FOR LOGISTIC REGRESSION
        for l in lambdas:
            Error_train_rlr_fold = []
            Error_test_rlr_fold = []
            error_inner = []
        # For each lambda we do the training with internal cross validation
            X_train_inner, y_train_inner = X_train[train_inner_idx], y_train[train_inner_idx]
            
            X_val_inner, y_val_inner = X_train[val_inner_idx], y_train[val_inner_idx]

            logreg = lm.LogisticRegression(C=1/l, solver="lbfgs", tol=1e-4, random_state=1)
            logreg.fit(X_train_inner, y_train_inner)
            
            error_val = np.mean(logreg.predict(X_val_inner) != y_val_inner)

        # If the error is less than the minimum error, we update the best lambda
            # if error_val < min_error:
            #     min_error = error_val
            #     best_lambda = l
                
            error_train_new = np.mean(logreg.predict(X_train_inner) != y_train_inner)
            Error_train_rlr_fold.append(error_train_new)
            Error_test_rlr_fold.append(error_val)
            
        #_____________________________________________________
        # FOR ANN
        # Define the model structure
        max_hidden_units = 5
        error_log = []
        for n_hidden_units in range(1, max_hidden_units):
            # number of hidden units in the signle hidden layer
            C = 3
            model = lambda: torch.nn.Sequential(
                torch.nn.Linear(M, n_hidden_units),  # M features to H hiden units
                torch.nn.ReLU(),  # 1st transfer function
                # Output layer:
                # H hidden units to C classes
                # the nodes and their activation before the transfer
                # function is often referred to as logits/logit output
                torch.nn.Linear(n_hidden_units, C),  # C logits
                # To obtain normalised "probabilities" of each class
                # we use the softmax-funtion along the "class" dimension
                # (i.e. not the dimension describing observations)
                torch.nn.Softmax(dim=1),  # final tranfer function, normalisation of logit output
            )
            # Since we're training a multiclass problem, we cannot use binary cross entropy,
            # but instead use the general cross entropy loss:
            loss_fn = torch.nn.CrossEntropyLoss()
            # Train the network:
            # C = 3
            net, _, _ = train_neural_net(
                model,
                loss_fn,
                X=torch.tensor(X_train, dtype=torch.float),
                y=torch.tensor(y_train, dtype=torch.long),
                n_replicates=3,
                max_iter=3000
            )
            # Determine probability of each class using trained network
            softmax_logits = net(torch.tensor(X_test, dtype=torch.float))
            # Get the estimated class as the class with highest probability (argmax on softmax_logits)
            y_test_est = (torch.max(softmax_logits, dim=1)[1]).data.numpy()
            # Determine errors
            e = y_test_est != y_test
            print(
                "Number of miss-classifications for ANN:\n\t {0} out of {1}".format(sum(e), len(e))
            )

            predict = lambda x: (
                torch.max(net(torch.tensor(x, dtype=torch.float)), dim=1)[1]
            ).data.numpy()
        #Find the best amount of hidden units
        best_hidden_units = range(1, max_hidden_units)[np.argmin(error_log)]
    
    #For ANN
    #_____________________________________________________
    # Average errors across folds for each hidden unit
    
    
    
    
    
    
    #FOR LAMBDA / LOGISTIC REGRESSION
    #_____________________________________________________
    # Average errors across folds for each lambda
    Error_test_avg.append(np.mean(Error_test_rlr_fold))
    
    best_lambda = lambdas[np.argmin(Error_test_avg)]
    
    # Train with best lambda on outer fold and evaluate amount of errors
    logreg_best = lm.LogisticRegression(C=best_lambda, solver="lbfgs",  tol=1e-4, random_state=1)
    logreg_best.fit(X_train, y_train)

    # Multinomial logistic regression
    logreg = lm.LogisticRegression(
        C=best_lambda,solver="lbfgs", multi_class="multinomial", tol=1e-4, random_state=1
    )
    logreg.fit(X_train, y_train)
    
    
    print("the best lambda is: ", best_lambda)
    
    print(lambdas)

    #_____________________________________________________
    
    # To display coefficients use print(logreg.coef_). For a 4 class problem with a
    # feature space, these weights will have shape (4, 2).

    # Number of miss-classifications
    # print(
    #     "Number of miss-classifications for Multinormal regression:\n\t {0} out of {1}".format(
    #         np.sum(logreg.predict(X_test) != y_test), len(y_test)
    #     )
    # )
    # print(logreg.predict(X_test))
    # print(y_test)
    
    # print("Coefficients")
    # print(logreg.coef_)
    # # Plotting should be done after the outer loop
    # plt.figure(figsize=(10, 6))
    # plt.semilogx(lambdas, Error_train_avg, 'b-', label='Training Error')
    # plt.semilogx(lambdas, Error_test_avg, 'r-', label='Test Error')
    # plt.xlabel('Lambda')
    # plt.ylabel('Error')
    # plt.legend()
    # plt.title('Training vs Test Error for Different Lambda Values')
    # plt.grid()
    # plt.show()
    # k += 1



	Replicate: 1/3
		Iter	Loss			Rel. loss
		1000	0.8610892	5.2258412e-05
		2000	0.8036066	8.098863e-05
		3000	0.7418473	4.4188484e-05
		Final loss:
		3000	0.7418473	4.4188484e-05

	Replicate: 2/3
		Iter	Loss			Rel. loss


/Users/lauge/Desktop/ML/MLenv/lib/python3.12/site-packages/dtuimldmtools/models/nn_trainer.py:141: RuntimeWarning: overflow encountered in cast
  if loss_value < best_final_loss:


		1000	0.8471376	8.1611004e-05
		2000	0.8091653	3.1305324e-05
		3000	0.77656454	6.209035e-05
		Final loss:
		3000	0.77656454	6.209035e-05

	Replicate: 3/3
		Iter	Loss			Rel. loss
		1000	0.83073884	5.165658e-05
		2000	0.7875844	6.326475e-05
		3000	0.72050035	5.8649817e-05
		Final loss:
		3000	0.72050035	5.8649817e-05
Number of miss-classifications for ANN:
	 0 out of 18

	Replicate: 1/3
		Iter	Loss			Rel. loss
		1000	0.6370824	0.00019578017
		2000	0.57613933	4.3863107e-05
		3000	0.56174624	1.40057955e-05
		Final loss:
		3000	0.56174624	1.40057955e-05

	Replicate: 2/3
		Iter	Loss			Rel. loss
		1000	0.6334299	0.00020114159
		2000	0.5776503	4.3542026e-05
		3000	0.562449	1.515397e-05
		Final loss:
		3000	0.562449	1.515397e-05

	Replicate: 3/3
		Iter	Loss			Rel. loss
		1000	0.62938833	0.00018103838
		2000	0.57801235	3.588448e-05
		3000	0.56598496	1.19000415e-05
		Final loss:
		3000	0.56598496	1.19000415e-05
Number of miss-classifications for ANN:
	 1 out of 18

	Replicate: 1/3
		Iter	Loss			

ValueError: attempt to get argmin of an empty sequence